In [3]:
import numpy as np
import pandas as pd
import re
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from joblib import dump, load

def generate_token_sent(s):
    try:
        s = s.lower()
    except:
        pass
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]
    sent = ""
    for i in range(len(tokens)):
        if i == 0:
            sent = sent + tokens[0]
        else:
            sent = sent + " " + tokens[i]
    return sent
    

def generate_token_sent_col(arr):
    new_arr = []
    for i in range(arr.size):
        new_arr.append(generate_token_sent(arr.iloc[i]))
    new_arr = np.array(new_arr)
    return new_arr

def generate_token_sent_col1(arr):
    new_arr = []
    for i in range(arr.size):
        new_arr.append(generate_token_sent(arr[i]))
    new_arr = np.array(new_arr)
    return new_arr

oll = pd.read_csv('all_data1.csv',encoding='utf-8')
for i in range(1):
    train_data, test_data = train_test_split(oll, test_size=0.01)
    train_data.columns = ['payload','type']
#     train_data.drop(['i'], axis=1,inplace=True)
    train_data.dropna(inplace=True)
    test_data.columns = ['payload','type']
#     test_data.drop(['i'], axis=1,inplace=True)
    test_data.dropna(inplace=True)
    train_data.loc[train_data['type'] == 'norm', 'class'] = 0
    train_data.loc[train_data['type'] == 'xss', 'class'] = 1
    test_data.loc[test_data['type'] == 'norm', 'class'] = 0
    test_data.loc[test_data['type'] == 'xss', 'class'] = 1
    
    trainpayload = train_data.payload
    trainresult = train_data['class']

    testpayload = test_data.payload
    testresult = test_data['class']
    
    trainpayload_new = generate_token_sent_col(trainpayload)
    testpayload_new = generate_token_sent_col(testpayload)
    
    count_vect = CountVectorizer()
    count_vect.fit(trainpayload_new)
    dump(count_vect, 'vectroizer.pkl')
    train_bow = count_vect.transform(trainpayload_new)
    test_bow = count_vect.transform(testpayload_new)
    
    clf = SVC(gamma='auto',kernel='linear',probability=True)
    clf.fit(train_bow, trainresult)
    dump(clf, 'svc.joblib') 
#     y_pred = clf.predict(test_bow)
#     print('Accuracy ~')
#     print(accuracy_score(testresult, y_pred))
    ele = '<script\x3Etype=\"text/javascript\">javascript:alert(1);</script>'
    testpayload = np.array([ele])
    testpayload_new = generate_token_sent_col1(testpayload)
    test_bow = count_vect.transform(testpayload_new)
    y_pred = clf.predict_proba(test_bow)
    print(y_pred)

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/root/anaconda3/lib/python3.7/site-packages/p

[[3.0000009e-14 1.0000000e+00]]
